In [1]:
import sqlite3
import pandas as pd
import os
import re
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
from nltk.tokenize import sent_tokenize, word_tokenize, StanfordTokenizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import string
import entropy
#%matplotlib inline 
#from textstat.textstat import textstat
#sns.set_style("whitegrid")
os.chdir('/Users/RayLJazz//Dropbox/UCDavis/2016spring/STA208/Github/Data')

# Read sql data

In [2]:
query = "SELECT * FROM [posts]"
conn = sqlite3.connect('crossvalidated.db')
post_df = pd.read_sql_query(query, conn)
post_df.head()
qpost_df = post_df[post_df.PostTypeId == 1]

In [3]:
qpost_df['Body'].shape

(75067,)

## Clean body
Remove:

1.Code lines 

2.links 

3.Emails 

4.Formulas

5.\newline, \tab

In [75]:
#Body_clean_puc= pd.read_pickle('Body_clean_puc.pickle')
def exact_from_html(row, puc=0):
    soup = BeautifulSoup(row, "html.parser")

    #remove code
    #chunck_num = len(soup.find_all('code'))
    #i=1
    #while True:
      #  if i <= chunck_num:
            #soup.code.extract()
            #i = i + 1
        #else:
            #break
    for tag in soup.find_all('code'):
        tag.replaceWith(' refcode ')
    #denote link
    for tag in soup.find_all('a'):
        content = tag.text
        tag.replaceWith(content +' (reflink) ')
    #denote image
    for tag in soup.find_all('img'):
        tag.replaceWith(' refimage ')
    ######
    raw = soup.get_text()
    #remove link
    raw_no_link = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", '', raw)
    #remove email
    no_link_email = re.sub(r'[\w\.-]+@[\w\.-]+[\.][com|org|ch|uk]{2,3}', "", raw_no_link)
    #remove whitespace
    tab_text = '\t\n\r\x0b\x0c'
    no_link_email_space = "".join([ch for ch in no_link_email if ch not in set(tab_text)])
    #remove math expression
    reg = '(\$.+?\$)|(\$\$.+?\$\$)|((\\\\begin\{.+?\})(.+?)(\\\\end\{(.+?)\}))'
    raw = re.sub(reg, "", no_link_email_space, flags=re.IGNORECASE)
    #remove numbers
    raw = re.sub('[0-9]+?', ' ', raw)
    #remove punctuation
    #if puc == 1:
     #   raw = "".join([ch for ch in raw if ch not in set(string.punctuation)])
    # remove stop words
    #raw = " ".join([word for word in word_tokenize(raw.lower()) if word not in stopwords.words('english')])
    return(raw)

In [76]:
fun_clean = lambda i: exact_from_html(i)
Body_clean_puc = qpost_df['Body'].map(fun_clean)

In [78]:
Body_clean_puc.head()

0    How should I elicit prior distributions from e...
1    In many different statistical methods there is...
2    What are some valuable Statistical Analysis op...
3    I have two groups of data.  Each with a differ...
5    Last year, I read a blog post from Brendan O'C...
Name: Body, dtype: object

In [79]:
import pickle
with open('Body_clean_puc1.pickle', 'wb') as handle:
    pickle.dump(Body_clean_puc, handle,protocol=2)

In [4]:
Body_clean_puc=pd.read_pickle('Body_clean_puc1.pickle')

## Exact code and formula

In [5]:
fomula_num = []
Code = []
Img=[]
def exact_formula(row):
    soup = BeautifulSoup(row, "html.parser")
    #remove code
    chunck_num = len(soup.find_all('code'))
    i=1
    while True:
        if chunck_num == 0:
            Code.append(0)
            break
        elif i <= chunck_num:
            code_num = 0
            code = soup.code.get_text()
            code = code.split('\n')
            codes = [x for x in code if x != '']
            code_num = code_num + len(codes)
            soup.code.extract()
            i = i + 1
        else:
            Code.append(code_num)
            break
    #remove img
    chunck_num = len(soup.find_all('img'))
    if chunck_num == 0:
        Img.append(0)
    else:
        Img.append(1)
    ######
    raw = soup.get_text()
    #remove link
    raw_no_link = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", '', raw)
    #remove email
    no_link_email = re.sub(r'[\w\.-]+@[\w\.-]+[\.][com|org|ch|uk]{2,3}', "", raw_no_link)
    #remove whitespace
    tab_text = '\t\n\r\x0b\x0c'
    no_link_email_space = "".join([ch for ch in no_link_email if ch not in set(tab_text)])
    # remove fomula
    reg = '(\$.+?\$)|(\$\$.+?\$\$)｜((\\\\begin\{.+?\})(.+?)(\\\\end\{(.+?)\}))'
    re_fomula = re.findall(reg, no_link_email_space, flags=re.IGNORECASE)
    if re_fomula != []:
        for item in re_fomula:
            no_link_email_space = no_link_email_space.replace(str(item), '')
        fomula_num.append(len(re_fomula))
    else:
        fomula_num.append(int(0))

In [6]:
fun_clean = lambda i: exact_formula(i)
get_result = qpost_df['Body'].map(fun_clean)

In [7]:
Code_df = pd.Series(data=Code)
Code_df.describe()
#Code_df.value_counts()

count    75067.000000
mean         2.250110
std         16.899923
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max       2160.000000
dtype: float64

In [8]:
#sns.distplot(Code_df, rug=False)

In [9]:
fomula_num_df = pd.Series(data=fomula_num)
fomula_num_df.value_counts().head()

0    54442
1     2364
2     1890
3     1840
4     1643
dtype: int64

In [10]:
pd.Series(data=Img).value_counts()

0    67585
1     7482
dtype: int64

In [11]:
Body_clean_puc_df = pd.DataFrame(Body_clean_puc)
Body_clean_puc_df['Code number'] = Code
Body_clean_puc_df['formula'] = fomula_num
Body_clean_puc_df['Imagine'] = Img
Body_clean_puc_df.head()

,Body,Code number,formula,Imagine
0,How should I elicit prior distributions from e...,0,0,0
1,In many different statistical methods there is...,0,0,0
2,What are some valuable Statistical Analysis op...,0,0,0
3,I have two groups of data. Each with a differ...,0,0,0
5,"Last year, I read a blog post from Brendan O'C...",0,0,0


In [12]:
#Body_clean_puc_df[Body_clean_puc_df['Code number']>50]

In [13]:
qpost_df['Code number'] = Code
qpost_df['formula'] = fomula_num
qpost_df['Imagine'] = Img
qpost_df.head()

/Users/RayLJazz/miniconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/RayLJazz/miniconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/RayLJazz/miniconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,Body,ViewCount,LastEditorDisplayName,ClosedDate,CommunityOwnedDate,LastEditorUserId,ParentID,LastEditDate,CommentCount,AnswerCount,...,PostTypeId,OwnerUserId,Tags,CreationDate,FavoriteCount,Id,LastActivityDate,Code number,formula,Imagine
0,<p>How should I elicit prior distributions fro...,1850.0,None,None,None,NaN,NaN,None,1,5.0,...,1,8.0,<bayesian><prior><elicitation>,2010-07-19T19:12:12.510,19.0,1,2010-09-15T21:08:26.077,0,0,0
1,<p>In many different statistical methods there...,15519.0,None,None,None,88.0,NaN,2010-08-07T17:56:44.800,1,7.0,...,1,24.0,<distributions><normality>,2010-07-19T19:12:57.157,9.0,2,2012-11-12T09:21:54.993,0,0,0
2,<p>What are some valuable Statistical Analysis...,5162.0,None,None,2010-07-19T19:13:28.577,183.0,NaN,2011-02-12T05:50:03.667,4,19.0,...,1,18.0,<software><open-source>,2010-07-19T19:13:28.577,38.0,3,2013-05-27T14:48:36.927,0,0,0
3,<p>I have two groups of data. Each with a dif...,15443.0,None,None,None,NaN,NaN,None,2,5.0,...,1,23.0,<distributions><statistical-significance>,2010-07-19T19:13:31.617,4.0,4,2010-09-08T03:00:19.690,0,0,0
5,"<p>Last year, I read a blog post from <a href=...",65657.0,None,None,2010-08-09T13:05:50.603,22047.0,NaN,2013-06-07T06:38:10.327,6,17.0,...,1,5.0,<machine-learning>,2010-07-19T19:14:44.080,211.0,6,2015-03-18T12:14:33.740,0,0,0


In [14]:
test_data = Body_clean_puc[1]

In [15]:
test_data

u'In many different statistical methods there is an "assumption of normality".  What is "normality" and how do I know if there is normality?'

# Readablity Function--Base

## Words count

In [16]:
# words count
def word_num(text):
    text = "".join(x for x in text if x not in list(set(string.punctuation)))
    text = word_tokenize(text)
    return (len(text))

In [17]:
word_num(test_data)

23

In [18]:
Words_num = Body_clean_puc_df['Body'].apply(word_num)
Words_num_df = pd.Series(data=Words_num)
Words_num_df.value_counts().head()

61    561
64    560
68    540
74    534
53    531
Name: Body, dtype: int64

## Sentence count

In [19]:
# sentence count
def sent_num(text):
    return (len(sent_tokenize(text)))

In [20]:
Sents_num = Body_clean_puc_df['Body'].apply(sent_num)
Sents_num_df = pd.Series(data=Sents_num)
Sents_num_df.value_counts().head()

3    10590
4    10181
2     9234
5     8798
6     6947
Name: Body, dtype: int64

## syllables count

In [21]:
# syllables count The "Written Method" Rules:http://www.howmanysyllables.com/howtocountsyllables
def syllable_num(text):
    count = 0
    vowels = 'aeiouy'
    text = text.lower()
    text = "".join(x for x in text if x not in list(set(string.punctuation)))

    if text == None:
        return 0
    elif len(text) == 0:
        return 0
    else:
        if text[0] in vowels: 
            count += 1
        for index in range(1, len(text)):
            if text[index] in vowels and text[index-1] not in vowels:
                count += 1
        if text.endswith('e'): 
            count -= 1
        if text.endswith('le'):
            count += 1
        if text.endswith('les'):
            count += 1
        if count == 0:
            count += 1
    return count

In [22]:
syllable_num(test_data)

43

## polysyllables count

In [23]:
# polysyllables count
def polysyllab_num(text):
    count = 0
    for word in text.split():
        wrds = syllable_num(word)
        if wrds >= 3:
            count += 1
    return count

In [24]:
polysyllab_num(test_data)

6

## Characters count

In [25]:
# characters count
def char_num(text, ignore_spaces=True):
    if ignore_spaces:
        text = text.replace(" ", "")
    text = "".join(x for x in text if x not in list(set(string.punctuation)))
    return len(text)

In [26]:
char_num(test_data)

109

## Complex words

In [27]:
easy_words = """a able aboard about above absent accept accident account ache aching acorn acre across act acts add address admire adventure afar	afraid after afternoon afterward afterwards again against age aged ago agree ah ahead aid aim air airfield airplane airport airship airy	alarm alike alive all alley alligator allow almost alone along aloud already also always am America American among amount an and	angel anger angry animal another answer ant any anybody anyhow anyone anything anyway anywhere apart apartment ape apiece appear apple April	apron are aren't arise arithmetic arm armful army arose around arrange arrive arrived arrow art artist as ash ashes aside ask	asleep at ate attack attend attention August aunt author auto automobile autumn avenue awake awaken away awful awfully awhile ax axe baa babe babies back background backward backwards bacon bad badge badly bag bake baker bakery baking ball balloon banana band bandage bang banjo bank banker bar barber bare barefoot barely bark barn barrel base baseball basement basket bat batch bath bathe bathing bathroom bathtub	battle battleship bay be beach bead beam bean bear beard beast beat beating beautiful beautify beauty became because become becoming bed bedbug bedroom bedspread bedtime bee beech beef beefsteak beehive been beer beet before beg began beggar begged begin beginning begun behave behind being	believe bell belong below belt bench bend beneath bent berries berry beside besides best bet better between bib bible bicycle bid big bigger bill billboard bin bind bird birth birthday biscuit bit bite biting bitter black blackberry blackbird blackboard blackness blacksmith blame blank blanket	blast blaze bleed bless blessing blew blind blindfold blinds block blood bloom blossom blot blow blue blueberry bluebird blush board boast boat bob bobwhite bodies body boil boiler bold bone bonnet boo book bookcase bookkeeper boom boot born borrow boss both bother bottle bottom	bought bounce bow bowl bow-wow box boxcar boxer boxes boy boyhood bracelet brain brake bran branch brass brave bread break breakfast breast breath breathe breeze brick bride bridge bright brightness bring broad broadcast broke broken brook broom brother brought brown brush bubble bucket buckle	bud buffalo bug buggy build building built bulb bull bullet bum bumblebee bump bun bunch bundle bunny burn burst bury bus bush bushel business busy but butcher butt butter buttercup butterfly buttermilk butterscotch button buttonhole buy buzz by bye cab cabbage cabin cabinet cackle cage cake calendar calf call caller calling came camel camp campfire can canal canary candle candlestick candy cane cannon cannot canoe can't canyon cap cape capital captain car card cardboard care careful careless carelessness carload carpenter carpet carriage carrot carry cart	carve case cash cashier castle cat catbird catch catcher caterpillar catfish catsup cattle caught cause cave ceiling cell cellar cent center cereal certain certainly chain chair chalk champion chance change chap charge charm chart chase chatter cheap cheat check checkers cheek cheer cheese cherry chest chew	chick chicken chief child childhood children chill chilly chimney chin china chip chipmunk chocolate choice choose chop chorus chose chosen christen Christmas church churn cigarette circle circus citizen city clang clap class classmate classroom claw clay clean cleaner clear clerk clever click cliff climb clip cloak	clock close closet cloth clothes clothing cloud cloudy clover clown club cluck clump coach coal coast coat cob cobbler cocoa coconut cocoon cod codfish coffee coffeepot coin cold collar college color colored colt column comb come comfort comic coming company compare conductor cone connect coo cook	cooked cooking cookie cookies cool cooler coop copper copy cord cork corn corner correct cost cot cottage cotton couch cough could couldn't count counter country county course court cousin cover cow coward cowardly cowboy cozy crab crack cracker cradle cramps cranberry crank cranky crash crawl crazy	cream creamy creek creep crept cried croak crook crooked crop cross crossing cross-eyed crow crowd crowded crown cruel crumb crumble crush crust cry cries cub cuff cup cuff cup cupboard cupful cure curl curly curtain curve cushion custard customer cut cute cutting dab dad daddy daily dairy daisy dam damage dame damp dance dancer dancing dandy danger dangerous dare dark darkness darling darn dart dash date daughter	dawn day daybreak daytime dead deaf deal dear death December decide deck deed deep deer defeat defend defense delight den dentist depend deposit describe desert	deserve desire desk destroy devil dew diamond did didn't die died dies difference different dig dim dime dine ding-dong dinner dip direct direction dirt dirty	discover dish dislike dismiss ditch dive diver divide do dock doctor does doesn't dog doll dollar dolly done donkey don't door doorbell doorknob doorstep dope	dot double dough dove down downstairs downtown dozen drag drain drank draw drawer draw drawing dream dress dresser dressmaker drew dried drift drill drink drip	drive driven driver drop drove drown drowsy drub drum drunk dry duck due dug dull dumb dump during dust dusty duty dwarf dwell dwelt dying each eager eagle ear early earn earth east eastern easy eat eaten	edge egg eh eight eighteen eighth eighty either elbow elder eldest electric	electricity elephant eleven elf elm else elsewhere empty end ending enemy engine	engineer English enjoy enough enter envelope equal erase eraser errand escape eve	even evening ever every everybody everyday everyone everything everywhere evil exact except	exchange excited exciting excuse exit expect explain extra eye eyebrow fable face facing fact factory fail faint fair fairy faith fake fall false family fan fancy far faraway fare farmer farm farming far-off farther fashion fast fasten fat father	fault favor favorite fear feast feather February fed feed feel feet fell fellow felt fence fever few fib fiddle field fife fifteen fifth fifty fig fight figure file fill	film finally find fine finger finish fire firearm firecracker fireplace fireworks firing first fish fisherman fist fit fits five fix flag flake flame flap flash flashlight flat flea flesh	flew flies flight flip flip-flop float flock flood floor flop flour flow flower flowery flutter fly foam fog foggy fold folks follow following fond food fool foolish foot football	footprint for forehead forest forget forgive forgot forgotten fork form fort forth fortune forty forward fought found fountain four fourteen fourth fox frame free freedom freeze freight French fresh	fret Friday fried friend friendly friendship frighten frog from front frost frown froze fruit fry fudge fuel full fully fun funny fur furniture further fuzzy gain gallon gallop game gang garage garbage garden gas gasoline gate gather gave gay gear geese general gentle gentleman gentlemen	geography get getting giant gift gingerbread girl give given giving glad gladly glance glass glasses gleam glide glory glove glow	glue go going goes goal goat gobble God god godmother gold golden goldfish golf gone good goods goodbye good-by goodbye	good-bye good-looking goodness goody goose gooseberry got govern government gown grab gracious grade grain grand grandchild grandchildren granddaughter grandfather grandma	grandmother grandpa grandson grandstand grape grapes grapefruit grass grasshopper grateful grave gravel graveyard gravy gray graze grease great green greet	grew grind groan grocery ground group grove grow guard guess guest guide gulf gum gun gunpowder guy ha habit had hadn't hail hair haircut hairpin half hall halt ham hammer hand handful handkerchief handle handwriting hang happen happily happiness happy harbor hard hardly hardship hardware hare hark	harm harness harp harvest has hasn't haste hasten hasty hat hatch hatchet hate haul have haven't having hawk hay hayfield haystack he head headache heal health healthy heap hear hearing	heard heart heat heater heaven heavy he'd heel height held hell he'll hello helmet help helper helpful hem hen henhouse her hers herd here here's hero herself he's hey hickory	hid hidden hide high highway hill hillside hilltop hilly him himself hind hint hip hire his hiss history hit hitch hive ho hoe hog hold holder hole holiday hollow holy	home homely homesick honest honey honeybee honeymoon honk honor hood hoof hook hoop hop hope hopeful hopeless horn horse horseback horseshoe hose hospital host hot hotel hound hour house housetop	housewife housework how however howl hug huge hum humble hump hundred hung hunger hungry hunk hunt hunter hurrah hurried hurry hurt husband hush hut hymn I ice icy I'd idea ideal if ill	I'll I'm important impossible improve in inch inches	income indeed Indian indoors ink inn insect inside	instant instead insult intend interested interesting into invite	iron is island isn't it its it's itself	I've ivory ivy jacket jacks jail jam January jar	jaw jay jelly jellyfish jerk jig	job jockey join joke joking jolly	journey joy joyful joyous judge jug	juice juicy July jump June junior	junk just keen keep kept kettle key	kick kid kill killed kind	kindly kindness king kingdom kiss	kitchen kite kitten kitty knee	kneel knew knife knit knives	knob knock knot know known lace lad ladder ladies lady laid lake lamb lame lamp land lane language lantern lap lard large lash lass last	late laugh laundry law lawn lawyer lay lazy lead leader leaf leak lean leap learn learned least leather leave leaving	led left leg lemon lemonade lend length less lesson let let's letter letting lettuce level liberty library lice lick lid	lie life lift light lightness lightning like likely liking lily limb lime limp line linen lion lip list listen lit	little live lives lively liver living lizard load loaf loan loaves lock locomotive log lone lonely lonesome long look lookout	loop loose lord lose loser loss lost lot loud love lovely lover low luck lucky lumber lump lunch lying machine machinery mad made magazine magic maid mail mailbox mailman major make making male mama mamma man manager mane manger many map	maple marble march March mare mark market marriage married marry mask mast master mat match matter mattress may May maybe mayor maypole me	meadow meal mean means meant measure meat medicine meet meeting melt member men mend meow merry mess message met metal mew mice middle	midnight might mighty mile milk milkman mill miler million mind mine miner mint minute mirror mischief miss Miss misspell mistake misty mitt mitten	mix moment Monday money monkey month moo moon moonlight moose mop more morning morrow moss most mostly mother motor mount mountain mouse mouth	move movie movies moving mow Mr. Mrs. much mud muddy mug mule multiply murder music must my myself nail name nap napkin narrow nasty naughty navy near nearby	nearly neat neck necktie need needle needn't Negro neighbor neighborhood	neither nerve nest net never nevermore new news newspaper next	nibble nice nickel night nightgown nine nineteen ninety no nobody	nod noise noisy none noon nor north northern nose not	note nothing notice November now nowhere number nurse nut oak oar oatmeal oats obey ocean o'clock October odd of off	offer office officer often oh oil old old-fashioned on once one	onion only onward open or orange orchard order ore organ other	otherwise ouch ought our ours ourselves out outdoors outfit outlaw outline	outside outward oven over overalls overcoat overeat overhead overhear overnight overturn	owe owing owl own owner ox pa pace pack package pad page paid pail pain painful paint painter painting pair pal palace pale pan pancake pane pansy pants papa paper parade pardon parent park part partly partner party	pass passenger past paste pasture pat patch path patter pave pavement paw pay payment pea peas peace peaceful peach peaches peak peanut pear pearl peck peek peel peep peg pen pencil penny	people pepper peppermint perfume perhaps person pet phone piano pick pickle picnic picture pie piece pig pigeon piggy pile pill pillow pin pine pineapple pink pint pipe pistol pit pitch pitcher pity	place plain plan plane plant plate platform platter play player playground playhouse playmate plaything pleasant please pleasure plenty plow plug plum pocket pocketbook poem point poison poke pole police policeman polish polite	pond ponies pony pool poor pop popcorn popped porch pork possible post postage postman pot potato potatoes pound pour powder power powerful praise pray prayer prepare present pretty price prick prince princess	print prison prize promise proper protect proud prove prune public puddle puff pull pump pumpkin punch punish pup pupil puppy pure purple purse push puss pussy pussycat put putting puzzle quack quart	quarter queen	queer question	quick quickly	quiet quilt	quit quite rabbit race rack radio radish rag rail railroad railway rain rainy rainbow raise raisin rake ram ran ranch rang rap rapidly	rat rate rather rattle raw ray reach read reader reading ready real really reap rear reason rebuild receive recess record red	redbird redbreast refuse reindeer rejoice remain remember remind remove rent repair repay repeat report rest return review reward rib ribbon rice	rich rid riddle ride rider riding right rim ring rip ripe rise rising river road roadside roar roast rob robber robe	robin rock rocky rocket rode roll roller roof room rooster root rope rose rosebud rot rotten rough round route row rowboat	royal rub rubbed rubber rubbish rug rule ruler rumble run rung runner running rush rust rusty rye sack sad saddle sadness safe safety said sail sailboat sailor saint salad sale salt same sand sandy sandwich sang sank sap sash sat satin satisfactory Saturday sausage savage save savings saw say scab scales scare scarf school schoolboy schoolhouse schoolmaster schoolroom scorch score scrap scrape scratch scream screen screw scrub sea seal seam search season seat second secret see seeing seed seek seem seen seesaw select self selfish	sell send sense sent sentence separate September servant serve service set setting settle settlement seven seventeen seventh seventy several sew shade shadow shady shake shaker shaking shall shame shan't shape share sharp shave she she'd she'll she's shear shears shed sheep sheet shelf shell shepherd shine shining shiny ship shirt shock shoe shoemaker shone shook shoot shop shopping shore short shot should shoulder shouldn't shout shovel show shower	shut shy sick sickness side sidewalk sideways sigh sight sign silence silent silk sill silly silver simple sin since sing singer single sink sip sir sis sissy sister sit sitting six sixteen sixth sixty size skate skater ski skin skip skirt sky slam slap slate slave sled sleep sleepy sleeve sleigh slept slice slid slide sling slip slipped slipper slippery slit slow slowly sly smack small smart smell	smile smoke smooth snail snake snap snapping sneeze snow snowy snowball snowflake snuff snug so soak soap sob socks sod soda sofa soft soil sold soldier sole some somebody somehow someone something sometime sometimes somewhere son song soon sore sorrow sorry sort soul sound soup sour south southern space spade spank sparrow speak speaker spear speech speed spell spelling spend spent spider spike spill spin spinach spirit spit	splash spoil spoke spook spoon sport spot spread spring springtime sprinkle square squash squeak squeeze squirrel stable stack stage stair stall stamp stand star stare start starve state station stay steak steal steam steamboat steamer steel steep steeple steer stem step stepping stick sticky stiff still stillness sting stir stitch stock stocking stole stone stood stool stoop stop stopped stopping store stork stories storm stormy story stove straight	strange stranger strap straw strawberry stream street stretch string strip stripes strong stuck study stuff stump stung subject such suck sudden suffer sugar suit sum summer sun Sunday sunflower sung sunk sunlight sunny sunrise sunset sunshine supper suppose sure surely surface surprise swallow swam swamp swan swat swear sweat sweater sweep sweet sweetness sweetheart swell swept swift swim swimming swing switch sword swore table tablecloth tablespoon tablet tack tag tail tailor take taken taking tale talk talker tall tame tan tank tap tape tar tardy task taste taught tax tea teach teacher team tear	tease teaspoon teeth telephone tell temper ten tennis tent term terrible test than thank thanks thankful Thanksgiving that that's the theater thee their them then there these they they'd they'll they're	they've thick thief thimble thin thing think third thirsty thirteen thirty this thorn those though thought thousand thread three threw throat throne through throw thrown thumb thunder Thursday thy tick ticket	tickle tie tiger tight till time tin tinkle tiny tip tiptoe tire tired title to toad toadstool toast tobacco today toe together toilet told tomato tomorrow ton tone tongue tonight too	took tool toot tooth toothbrush toothpick top tore torn toss touch tow toward towards towel tower town toy trace track trade train tramp trap tray treasure treat tree trick tricycle tried	trim trip trolley trouble truck true truly trunk trust truth try tub Tuesday tug tulip tumble tune tunnel turkey turn turtle twelve twenty twice twig twin two ugly umbrella uncle under understand underwear	undress unfair unfinished unfold unfriendly unhappy	unhurt uniform United States unkind unknown	unless unpleasant until unwilling up upon	upper upset upside upstairs uptown upward	us use used useful valentine valley valuable	value vase vegetable	velvet very vessel	victory view village	vine violet visit	visitor voice vote wag wagon waist wait wake waken walk wall walnut want war warm warn was wash washer washtub wasn't waste watch watchman water watermelon waterproof wave wax	way wayside we weak weakness weaken wealth weapon wear weary weather weave web we'd wedding Wednesday wee weed week we'll weep weigh welcome well went were	we're west western wet we've whale what what's wheat wheel when whenever where which while whip whipped whirl whisky whiskey whisper whistle white who who'd whole	who'll whom who's whose why wicked wide wife wiggle wild wildcat will willing willow win wind windy windmill window wine wing wink winner winter wipe wire	wise wish wit witch with without woke wolf woman women won wonder wonderful won't wood wooden woodpecker woods wool woolen word wore work worker workman world	worm worn worry worse worst worth would wouldn't wound wove wrap wrapped wreck wren wring write writing written wrong wrote wrung yard yarn year yell	yellow yes yesterday yet	yolk yonder you you'd	you'll young youngster your	yours you're yourself yourselves	youth you've"""
easy_word_set = set(easy_words.split())
def complex_words_num(text):
	text_list = text.split()
	diff_words_set = set()
	for value in text_list:
		if value not in easy_word_set:
			if syllable_num(value) > 1:
				if value not in diff_words_set:
					diff_words_set.add(value)
	return len(diff_words_set)

In [28]:
complex_words_num(test_data)

6

## Some useful function

In [29]:
def word_sent(text): #avg_sentence_length
    lc = word_num(text)
    sc = sent_num(text)
    return float(lc)/float(sc)


def syll_word(text): #avg_syllables_per_word
    syllable = syllable_num(text)
    words = word_num(text)
    ASPW = float(syllable)/float(words)
    return ASPW



def char_word(text): #avg_letter_per_word
    ALPW = float(float(char_num(text))/float(word_num(text)))
    return ALPW



def word_sent(text):
    AWPS = float(float(word_num(text))/float(sent_num(text)))
    return AWPS

def sent_word(text): #avg_sentence_per_word
    ASPW = float(float(sent_num(text))/float(word_num(text)))
    return ASPW


In [30]:
text=test_data

In [31]:
#word_sent(text)
#syll_word(text)
#char_word(text)
#sent_word(text)

# Readablity Function--Advance

## Automated Reading Index

In [32]:
def Automated_Readability_Index(text):
    try:
        a = char_word(text)
        b = word_sent(text)
        ARI = (4.71 * round(a, 2)) + (0.5*round(b, 2)) - 21.43
        return round(ARI, 2)
    except:
        return 'NaN'

In [33]:
Automated_Readability_Index(text)

6.65

In [34]:
ARI = Body_clean_puc_df['Body'].apply(Automated_Readability_Index)

In [35]:
ARI.value_counts().head()

9.70     93
10.63    93
10.23    92
10.21    92
8.80     89
Name: Body, dtype: int64

## Coleman Liau Index

In [36]:
def Coleman_Liau_Index(text):
    try:
        L = char_word(text)*100
        S = sent_word(text)*100
        CLI = float((0.058 * L) - (0.296 * S) - 15.8)
        return round(CLI, 2)
    except:
        return 'NaN'

In [37]:
Coleman_Liau_Index(test_data)

9.11

In [38]:
CLI = Body_clean_puc_df['Body'].apply(Coleman_Liau_Index)

In [39]:
CLI.value_counts().head()

10.27    157
11.23    146
9.10     142
10.13    139
10.53    139
Name: Body, dtype: int64

## Flesch Kincaid Grade Level

In [40]:
def Flesch_Kincaid_Grade(text):
    try:
        ASL = word_sent(text)
        ASW = syll_word(text)
        FKRA = float(0.39 * ASL) + float(11.8 * ASW) - 15.59
        return round(FKRA, 2)
    except:
        return 'NaN'

In [41]:
Flesch_Kincaid_Grade(text)

10.96

In [42]:
FKG = Body_clean_puc_df['Body'].apply(Flesch_Kincaid_Grade)

In [43]:
FKG.value_counts().head()

12.83    154
11.68    150
11.08    139
13.90    139
12.27    137
Name: Body, dtype: int64

## Flesch reading ease

In [44]:
def Flesch_Reading_Ease(text):
    try:
        ASL = word_sent(text)
        ASW = syll_word(text)
        FRE = 206.835 - float(1.015 * ASL) - float(84.6 * ASW)
        return round(FRE, 2)
    except:
        return 'NaN'

In [45]:
Flesch_Reading_Ease(text)

37.0

In [46]:
FRE = Body_clean_puc_df['Body'].apply(Flesch_Reading_Ease)

In [47]:
FRE.value_counts().head()

39.33    100
35.95     90
47.83     87
43.96     67
36.46     67
Name: Body, dtype: int64

## Gunning fog index

In [48]:
def Gunning_Fog(text):
    try:
        per_diff_words = (complex_words_num(text)/word_num(text)*100)
        grade = 0.4*(word_sent(text) + per_diff_words)
        return round(grade,2)
    except:
        return 'NaN'

In [49]:
Gunning_Fog(text)

4.6

In [50]:
GFI = Body_clean_puc_df['Body'].apply(Gunning_Fog)

In [51]:
GFI.value_counts().head()

6.8    864
7.6    864
6.4    847
7.2    843
6.0    824
Name: Body, dtype: int64

## SMOG Index

In [75]:
import math

In [77]:
def SMOG_Index(text):
    try:
        grade = 1.043*math.sqrt(polysyllab_num(text)*30/sent_num(text))+3.1291
        return round(grade,2)
    except:
        return 'NaN'

In [78]:
SMOG = Body_clean_puc_df['Body'].apply(SMOG_Index)

In [79]:
SMOG.value_counts().head()

13.02    3878
11.21    3331
14.55    3296
15.90    2438
12.16    1943
Name: Body, dtype: int64

## Metric Entropy

In [55]:
def Metric_Entropy(original): #original body with tags
    try:
        sentropy = entropy.shannon_entropy(original)
        body_length = len(original)
        ME = float(sentropy/body_length)
        return ME
    except:
        return 'NaN'

In [56]:
Metric_Entropy(qpost_df['Body'][1])

0.0036979012642914828

In [57]:
ME = qpost_df['Body'].apply(Metric_Entropy)

In [58]:
ME.value_counts().head()

NaN                  5249
0.00134663514109        2
0.000431743403253       2
0.000899450906126       2
0.00114828761518        2
Name: Body, dtype: int64

## LOC Percentage

In [59]:
Body_clean_puc_df['sentence_count'] = Sents_num

In [60]:
def LOC_ptg(data):
    try:
        LOC = float(data['Code number'])
        LOB = float(data['sentence_count'])
        LP = LOC/(LOC+LOB)
        return LP
    except:
        return 'NaN'

In [61]:
LOC_ptg(Body_clean_puc_df[73923:73924])

0.6666666666666666

In [62]:
LOC_Per = Body_clean_puc_df.apply(LOC_ptg, axis=1)
#LOC_Per = []
#for index, row in Body_clean_puc_df.iterrows():
 #   LOC_Per.append(LOC_ptg(row))

In [63]:
LOC_Per.value_counts()

0.000000    56114
0.250000     1209
0.333333     1168
0.200000     1149
0.500000     1089
0.166667     1063
0.142857      834
0.125000      663
0.111111      527
0.666667      486
0.100000      419
0.090909      327
0.600000      311
0.400000      306
0.750000      267
0.083333      247
0.571429      231
0.076923      224
0.428571      215
0.800000      185
0.625000      179
0.285714      175
0.444444      159
0.071429      156
0.555556      154
0.714286      151
0.833333      148
0.375000      145
0.066667      121
0.700000      116
            ...  
0.680851        1
0.978378        1
0.957219        1
0.872340        1
0.924242        1
0.487805        1
0.002457        1
0.475000        1
0.018868        1
0.932203        1
0.009009        1
0.527778        1
0.010000        1
0.968085        1
0.919540        1
0.044776        1
0.628571        1
0.902778        1
0.997691        1
0.934783        1
0.982843        1
0.901408        1
0.322581        1
0.130435        1
0.728814  

In [64]:
Body_clean_puc_df['LOC_Per'] = LOC_Per

In [65]:
Body_clean_puc_df[73923:73924]

,Body,Code number,formula,Imagine,sentence_count,LOC_Per
149316,I have a x transplant experiment. There ar...,20,0,0,10,0.666667


# Generate pickle

In [80]:
Body_clean_puc_df['Id'] = qpost_df['Id']
Body_clean_puc_df['word_count'] = Words_num
Body_clean_puc_df['sentence_count'] = Sents_num
Body_clean_puc_df['ARI'] = ARI
Body_clean_puc_df['CLI'] = CLI
Body_clean_puc_df['FKG'] = FKG
Body_clean_puc_df['FRE'] = FRE
Body_clean_puc_df['GFI'] = GFI
Body_clean_puc_df['SMOG'] = SMOG
Body_clean_puc_df['M_Entropy'] = ME
Body_clean_puc_df['LOC_Per'] = LOC_Per
Body_clean_puc_df['Score'] = qpost_df['Score']

In [81]:
Body_clean_puc_df.head()

,Body,Code number,formula,Imagine,sentence_count,LOC_Per,Id,word_count,ARI,CLI,FKG,FRE,GFI,SMOG,M_Entropy,Score
0,How should I elicit prior distributions from e...,0,0,0,1,0.0,1,13,10.41,13.15,9.45,50.47,5.20,11.21,0.00613826,31
1,In many different statistical methods there is...,0,0,0,2,0.0,2,23,6.65,9.11,10.96,37.00,4.60,13.02,0.0036979,26
2,What are some valuable Statistical Analysis op...,0,0,0,1,0.0,3,29,18.55,14.58,18.51,14.03,11.60,17.12,0.00275086,63
3,I have two groups of data. Each with a differ...,0,0,0,5,0.0,4,81,9.04,9.94,10.98,45.21,6.48,13.02,0.00110337,15
5,"Last year, I read a blog post from Brendan O'C...",0,0,0,7,0.0,6,185,17.17,14.34,16.47,24.07,10.57,15.51,0.000372775,226


In [82]:
import pickle
with open('Readability_Metrics_new.pickle', 'wb') as handle:
    pickle.dump(Body_clean_puc_df, handle,protocol=2)

In [83]:
target = ['Id','Code number','formula','Imagine','word_count','sentence_count','ARI','CLI','FKG','FRE','GFI','SMOG','M_Entropy','LOC_Per','Score']
RM = Body_clean_puc_df[target]
RM.head()

,Id,Code number,formula,Imagine,word_count,sentence_count,ARI,CLI,FKG,FRE,GFI,SMOG,M_Entropy,LOC_Per,Score
0,1,0,0,0,13,1,10.41,13.15,9.45,50.47,5.20,11.21,0.00613826,0.0,31
1,2,0,0,0,23,2,6.65,9.11,10.96,37.00,4.60,13.02,0.0036979,0.0,26
2,3,0,0,0,29,1,18.55,14.58,18.51,14.03,11.60,17.12,0.00275086,0.0,63
3,4,0,0,0,81,5,9.04,9.94,10.98,45.21,6.48,13.02,0.00110337,0.0,15
5,6,0,0,0,185,7,17.17,14.34,16.47,24.07,10.57,15.51,0.000372775,0.0,226


In [84]:
import pickle
with open('Readability_Metrics_wtbody_new.pickle', 'wb') as handle:
    pickle.dump(RM, handle,protocol=2)